# 动态属性和特性

属性（attribute）：数据的属性以及处理数据的方法的总称
特性（property）：不改变类接口的前提下，使用存取方法修改数据属性的方法


此外，Python还提供了控制属性以及实现动态属性的API。

本章以处理嵌套字典为例说明如何在Python中实现动态属性。

## 基于动态属性的嵌套字典访问

访问嵌套字典中某个特定数据最简单的方法就是通过多个键来访问，类似于：test_dict[key_1][key_2][key_3]，这种访问方法略显臃肿。另一种方法就是动态属性，即，运行使用类似于访问属性的方式访问嵌套字典中某个特定数据，类似于：test_dict.key_1.key_2.key_3

实现上述功能的关键是定义__getattr__方法，该方法会在无法使用常规方法获取属性时被调用

本章实现的嵌套字典访问类如下，由于该类仅实现了__init__和__getattr__，因此尝试获取其他属性时均会调用__getattr__

对其逻辑的分析如下：
1. 将字典转换为FrozenDict对象
2. 当使用FrozenDict.key访问某一个属性时
    * self._data具有该属性（注意是属性，而不是key），直接获取该属性并返回
    * self._data不具有该属性，尝试从self._data[key]建立新FrozenDict并返回

对于build类方法：
1. 若self._data[key]是映射（字典），则利用该映射创建新FrozenDict
2. 若self._data[key]是序列（列表），则尝试利用该序列的元素逐个创建新FrozenDict
3. 若既不是映射也不是序列（大概率是数值、字符串），则直接返回该元素

In [1]:
from collections import abc

class FrozenDict:

    def __init__(self, mapping):
        self._data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self._data, name):
            return getattr(self._data, name)
        else:
            return FrozenDict.build(self._data[name])
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

上述创建的FrozenDict有如下几个特点（缺陷）：
1. 没有对每一次访问的结果进行缓存或转换保存，每次调用均是临时创建
2. 没有对key的名称进行特殊处理，若存在以Python关键字命名的key，会出现无法调用的情况
3. key不是有效的标识符（例如包含特殊字符或者以数字开头），因此无法调用

对于第一点，当传入的dict不是非常复杂时无关紧要；对于第二点，可以尝试使用对键名进行改写来规避这个问题；对于第三点，则需要对键名进行完全的改写或者直接抛出错误拒绝处理

### __init__和__new__

上述嵌套字典的解析依靠类方法build。build类方法的存在使得访问实例属性时，不同的值会返回不同类型的对象。

Python中创建类实例的实际上是__new__，创建的实例会作为第一个参数传递给__init__，然后__init__实现初始化。__new__默认返回的是当前类的实例，__new__实际上也可以返回其他类的实例，此时解释器不会调用__init__方法。

鉴于此，上述的嵌套字典解析类可以使用__new__代替build，代码如下：

In [ ]:
from collections import abc

class FrozenDict:

    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return cls(arg)
        elif isinstance(arg, abc.MutableSequence):
            return [cls.build(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self._data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self._data, name):
            return getattr(self._data, name)
        else:
            return FrozenDict(self._data[name])

## 特性与属性验证

默认情况下，可以任意改变Python属性的取值，但是在一些应用中取值有一些限制。为了施加这种限制，一种朴素的方法是：分离读值方法以及设值方法，并且在设值方法中对值进行验证。

本书以一个简单的订货系统为例，描述上述的过程

基础代码如下

In [ ]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

在该实现中，并没有对属性的取值进行验证，当设值是无效时，该类的实例也会返回错误的结果（正常来说，这里应当拒绝计算或者阻止设置无效值）。

基于此，可以将上述类改写如下。

In [6]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description

        # 此处就会调用设值方法对weight进行检查
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    @property
    # 读值方法
    def weight(self):
        return self._weight
    
    @weight.setter
    # 设值方法
    def weight(self, value):
        # 属性验证
        if value > 0:
            self._weight = value
        else:
            raise ValueError("value must be > 0")
    
test_item = LineItem("test", -10, 1)

ValueError: value must be > 0

### property

property实际上是一个类，虽然常用作装饰器

其构造方法如下：
property(fget=None, fset=None, fdel=None, doc=None)

所有参数都是可选的，分别实现：读值方法、设值方法、删除方法以及文档

### 特性、实例属性以及类属性

主要是几个规则，对于同名的属性/特性：
1. 实例属性会遮盖类属性
2. 实例属性不会覆盖类特性（尝试覆盖则抛出AttributeError）
3. 对类进行操作可以实现特性的覆盖以及销毁
4. 新增的类特性会遮盖现有的实例属性，销毁后复原

上述规则实际上表明：特性的“优先级”高于属性，当尝试获取attr时，首先会从obj.__class__中查找是否有名为attr的特性

### 文档

property的构造方法表明特性是可以设定文档的，当以装饰器的形式创建property时，该方法的文档字符串会被返回，测试例子如下

In [10]:
class Foo:

    @property
    def bar(self):
        """
        The bar attribute
        """
        return self.__dict__["bar"]
    
    @bar.setter
    def bar(self, value):
        self.__dict__["bar"] = value

print("Foo类的帮助文档")
help(Foo)

print("\nFoo.bar特性的帮助文档")
help(Foo.bar)

Foo类的帮助文档
Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute


Foo.bar特性的帮助文档
Help on property:

    The bar attribute



## 特性工厂

通常，一个类中可能有多个属性需要设定读值以及设值方法，若使用上述的方法逐个设置，显然有大量冗余代码。本节就是尝试对此进行改进，具体来说，使用“特性工厂”封装读值以及设值。

本节依然以上述的订货系统为例，在下述实现中weight和price均要求>0

In [7]:
def quantity(storage_name):

    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError("value must be > 0")
    
    return property(qty_getter, qty_setter)

class LineItem:

    weight = quantity("weight")
    price = quantity("price")

    def __init__(self, description, weight, price):
        self.description = description
        # 此处就会调用设值方法对weight进行检查
        self.weight = weight
        # 此处就会调用设值方法对price进行检查
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

test_item = LineItem("test", 10, -1)

ValueError: value must be > 0

上述代码实际上规避了一些潜在问题，具体来说：
1. quantity中尝试直接从obj.__dict__中取值或者存值，避免特性的无限递归（若直接使用obj.attr存取值，则会无限递归特性，因为在调用属性前始终会检查是否有同名的特性）
2. 同样得益于quantity的实现方法，上述LineItem不再需要使用_weight和_price存储以及设定对应的self.weight和self.price的值

## 属性删除

类似于设值以及传值，特性同样可以用来定义如何删除属性。具体来说，就是传入property(fget=None, fset=None, fdel=None, doc=None)中的fdel。

## 处理属性的重要属性和函数

本节主要是汇总了处理属性时会涉及到的属性以及函数，需要重点关注

### 特殊属性

* __class__属性
对象所属类的引用，类属性以及特性均会在该属性中寻找
* __dict__属性
存储对象或者类的可写属性
* __slots__属性
若存在__slots__则实例可能没有__dict__，该属性用于指明对象允许拥有的属性

### 内置函数

* dir([object])
列出对象的大多数属性
* getattr(object, name[, default])
从object中获取name对应的属性
* hasattr(object, name)
判断object是否有名为name的属性
* setattr(object, name, value)
设定object的name属性的值为value
* vars([object])
返回object的__dict__属性，若不存在__dict__则抛出错误

### 特殊方法

有两点注意：
1. 通常假定特殊方法从类获取，即特殊方法不会被同名的实例属性覆盖
2. 通过内置函数存取属性时会触发对应的特殊方法，为了绕开这些特殊方法，可以直接读写实例的__dict__

* __delattr__(self, name)
删除属性 ——> del语句
* __dir__(self)
列出属性 ——> dir函数
* __getattr__(self, name)
通常方法获取属性失败后调用（obj、Class以及SubClass中均找不到该属性时触发）
* __getattribute__(self, name)
获取属性，若抛出AttributeError时调用__getattr__方法
* __setattr__(self, name, value)
设定属性值

## 总结

* Python中属性（attribute）和特性（property）是两个概念，前者是数据属性以及处理数据的方法的总称，后者则是在不改变类接口的前提下，使用存取方法修改数据属性的方法
* 由于在通过通用方法没有查询到特定属性的值时均会调用__getattr__，通过实现__getattr__方法能够实现非常灵活的动态属性存取
* __new__和__init__有千丝万缕的联系，Python中创建实例的实际上是__new__，实现初始化的则是__init__
* 分离读值方法和设值方法是为属性取值施加约束的朴素方法，Python的特性为实现这一功能提供了很好的支持
* @property装饰器以及@attr.setter装饰器能够分别实现属性的读值方法以及设值方法
* property通常作为装饰器出现，但是其实际上是一个类，其构造方法为：property(fget=None, fset=None, fdel=None, doc=None)，即允许定义读值方法、设值方法、删除方法以及文档
* 特性、实例属性以及类属性之间有严格的处理顺序，其中特性总是被优先处理
* 为了实现上述的动态属性编程，Python从特殊属性、内置函数以及特殊方法三个方面提供了支持